In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pickle
import astropy.coordinates as coord
import astropy.units as u
from astropy.io import ascii

In [2]:
CAT = pickle.load(open('./pickle/cat236_257.pk','rb'))

In [3]:
master_cat = pd.read_csv('./catdata/master_gc_gals_catalog.csv')

In [4]:
master_cat

,ID,RADEG,DECDEG,CLASS
0,B001,9.962537,40.969619,1.0
1,B002,10.010729,41.198203,1.0
2,B003,10.039204,41.184903,1.0
3,B004,10.074675,41.377858,1.0
4,B005,10.084663,40.732864,1.0
...,...,...,...,...
2179,dTZZ-C65,22.822500,45.728611,4.0
2180,dTZZ-C66,23.190833,42.958889,4.0
2181,dTZZ-C67,23.496250,42.633889,4.0
2182,dTZZ-C68,23.525000,45.725556,4.0


Max and min of each field

In [5]:
bounds = {}

In [6]:
# define the bounds of each field
# (x0,y0) -> (x1,y1) is the bounding box of the field
# note that since RA decreases in the easterly direction, we take x1 to be the minimum value of RA
for c in CAT:
    x0 = max(CAT[c]['RA'])
    x1 = min(CAT[c]['RA'])
    y0 = min(CAT[c]['Dec'])
    y1 = max(CAT[c]['Dec'])
    bounds[c] = (x0,x1,y0,y1)

In [7]:
# match objects in CAT[n] to the ones in master_cat
def match_objs(cat_no):
    d1 = CAT[cat_no]
    ra_obs, dec_obs = d1['RA'], d1['Dec']
    d2 = master_cat
    ra_qry, dec_qry = d2['RADEG'], d2['DECDEG']

    c1 = coord.SkyCoord(ra_obs, dec_obs, unit=(u.degree, u.degree))
    c2 = coord.SkyCoord(ra_qry, dec_qry, unit=(u.degree, u.degree))
    idx, d2d, _ = c1.match_to_catalog_sky(c2)
    return idx,d2d

In [8]:
# find a list of all objects from match_objs that are actually contained within a given field
def find_objs_in_field(cat_no,objs):
    in_field = []
    for o in objs:
        ra = master_cat.loc[o]['RADEG']
        dec = master_cat.loc[o]['DECDEG']
        x0,x1,y0,y1 = bounds[cat_no]
        if ( ra<x0 and ra>x1 and dec>y0 and dec<y1 ):
            in_field.append(master_cat.loc[o])
    return in_field

In [9]:
# find the closest match(es) for each object in the master_list, that is in a given field
def find_closest_match(cat_no, in_field, delta):
    ret = []
    for obj in in_field:
        idx = obj.name #obj.name is the position/index of the object in master_cat
        closest_obj_filter = d2d_idx_df[d2d_idx_df['idx']==idx]['d2d'] < delta
        closest_obj_idx = d2d_idx_df[d2d_idx_df['idx']==idx]['d2d'].idxmin() #!!!!!!!!!
        
        d2d_idx_df[d2d_idx_df['idx']==idx][closest_obj_filter] # filter from the true/false table for dist within delta
        
        out = (closest_obj_idx,idx) # return index of object in CAT[n] and index in master_cat
        ret.append(out)
    return ret

In [10]:
idx, d2d = match_objs(253)

In [11]:
d2d = d2d.degree

In [12]:
d2d_idx_tuples = [(d2d[i],idx[i]) for i in range(len(d2d))]

In [13]:
d2d_idx_df = pd.DataFrame(d2d_idx_tuples, columns=['d2d','idx'])

In [19]:
objs=set(idx)

In [21]:
in_field = find_objs_in_field(253,objs)

In [44]:
filter1 = d2d_idx_df[d2d_idx_df['idx']==1220]['d2d'] < 0.005 # what value of delta???????? how much RA/Dec dist is allowed

In [45]:
d2d_idx_df[d2d_idx_df['idx']==1220][filter1]

,d2d,idx
22614,0.000026,1220
22707,0.001086,1220
22742,0.001122,1220
22797,0.001345,1220
22824,0.001396,1220
...,...,...
27003,0.004757,1220
27142,0.003100,1220
28085,0.004396,1220
28291,0.004913,1220


In [16]:
find_closest_match(253, in_field)

[(195123, 1184), (92796, 1219), (22614, 1220), (86008, 1214)]

In [ ]:
# main loop

for c in CAT:
    idx,d2d = match_objs(c)
    objs = set(idx)
    d2d = d2d.deg
    
    d2d_idx_tuples = [(d2d[i],idx[i]) for i in range(len(d2d))]
    
    d2d_idx_df = pd.DataFrame(d2d_idx_tuples, columns=['d2d','idx'])
    
    # get the list of all obj from master_cat which are in the field CAT[c]
    in_field = find_objs_in_field(c,objs)
    matches = find_closest_match(c, in_field)

In [41]:
objs = set(idx)

In [54]:
master_cat.loc[2138]

ID        dTZZ-SDSS7
RADEG       11.92125
DECDEG     42.071389
CLASS            4.0
Name: 2138, dtype: object